<a href="https://colab.research.google.com/github/beggy22/modulo_10-connecting-to-a-sql-database-project-tutorial/blob/main/SQL_con_SQLITE_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import os
from sqlalchemy import create_engine, text
import sqlite3

# Configuración del nombre de la base de datos
db_name = 'mydatabase.db'

# Eliminar la base de datos si existe
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Base de datos {db_name} eliminada.")
else:
    print(f"No se encontró la base de datos {db_name}.")

# Crear la conexión a la base de datos
engine = create_engine(f'sqlite:///{db_name}')
print("Ejecutando sentencias de creación de tablas...")

# Ejecutar sentencias de creación de tablas
with engine.connect() as connection:
    with open('create.sql') as file:
        create_sql = file.read()
        statements = create_sql.split(';')
        for statement in statements:
            if statement.strip():
                try:
                    connection.execute(text(statement))
                    print(f"Ejecución exitosa de la sentencia: {statement.strip()}")
                except Exception as e:
                    print(f"Error al ejecutar la sentencia: {statement.strip()}")
                    print(e)

print("Tablas creadas en la base de datos.")
print("Insertando datos en las tablas...")

# Insertar datos en las tablas
with engine.connect() as connection:
    trans = connection.begin()  # Iniciar una transacción
    try:
        with open('insert.sql') as file:
            insert_sql = file.read()
            statements = insert_sql.split(';')
            for statement in statements:
                if statement.strip():
                    connection.execute(text(statement))
                    print(f"Ejecución exitosa de la sentencia: {statement.strip()}")
        trans.commit()  # Confirmar la transacción
    except Exception as e:
        trans.rollback()  # Revertir la transacción en caso de error
        print(f"Error al ejecutar la sentencia: {statement.strip()}")
        print(e)

print("Datos insertados en las tablas.")

# Verificar los datos en todas las tablas
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Obtener el listado de todas las tablas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Consultar y mostrar los datos de cada tabla
for table in tables:
    table_name = table[0]
    print(f"\nDatos en la tabla {table_name}:")
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    if rows:
        for row in rows:
            print(row)
    else:
        print(f"No se encontraron datos en la tabla {table_name}.")

cursor.close()
conn.close()

Base de datos mydatabase.db eliminada.
Ejecutando sentencias de creación de tablas...
Ejecución exitosa de la sentencia: CREATE TABLE publishers(
    publisher_id INT NOT NULL,
    name VARCHAR(255) NOT NULL,
    PRIMARY KEY(publisher_id)
)
Ejecución exitosa de la sentencia: CREATE TABLE authors(
    author_id INT NOT NULL,
    first_name VARCHAR(100) NOT NULL,
    middle_name VARCHAR(50) NULL,
    last_name VARCHAR(100) NULL,
    PRIMARY KEY(author_id)
)
Ejecución exitosa de la sentencia: CREATE TABLE books(
    book_id INT NOT NULL,
    title VARCHAR(255) NOT NULL,
    total_pages INT NULL,
    rating DECIMAL(4, 2) NULL,
    isbn VARCHAR(13) NULL,
    published_date DATE,
    publisher_id INT NULL,
    PRIMARY KEY(book_id),
    CONSTRAINT fk_publisher FOREIGN KEY(publisher_id) REFERENCES publishers(publisher_id)
)
Ejecución exitosa de la sentencia: CREATE TABLE book_authors (
    book_id INT NOT NULL,
    author_id INT NOT NULL,
    PRIMARY KEY(book_id, author_id),
    CONSTRAINT fk_